In [1]:
import sys
import torch
import numpy as np

print("Python version:", sys.version)
print('torch', torch.__version__)
print('numpy', np.__version__)

import itertools

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from text_loader import TextDataset

#따로 생성된 py파일
import seq2seq_models as sm
from seq2seq_models import str2tensor, EOS_token, SOS_token

Python version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
torch 2.2.0+cpu
numpy 1.19.5


# seq to seq

In [49]:
HIDDEN_SIZE = 128
N_LAYERS = 1
BATCH_SIZE = 3
N_EPOCH = 20
N_CHARS = 128  # ASCII

In [50]:
def test():
    encoder_hidden = encoder.init_hidden()
    word_input = str2tensor('hello')
    encoder_outputs, encoder_hidden = encoder(word_input, encoder_hidden)
    print(encoder_outputs)
    
    decoder_hidden = encoder_hidden
    
    word_target = str2tensor('pytorch')
    for cc in range(len(word_target)):
        decoder_output, decoder_hidden = decoder(word_target[cc], decoder_hidden)
    print(decoder_output.size(), decoder_hidden.size())
    
def train(src, target):
    src_var = str2tensor(src)
    target_var = str2tensor(target, eos = True)
    
    encoder_hidden = encoder.init_hidden()
    encoder_outputs, encoder_hidden = encoder(src_var, encoder_hidden)
    
    hidden = encoder_hidden
    loss = 0
    
    for cc in range(len(target_var)):
        token = target_var[cc - 1] if cc else str2tensor(SOS_token)
        output, hidden = decoder(token, hidden)
        
        
        predicted_char_index = torch.argmax(output)
        target_char_index = torch.tensor(target_var[cc])
        loss += criterion(output.view(1, -1), target_char_index.view(1))
        
        decoder_in = target_char_index
        
    encoder.zero_grad()
    decoder.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item() / len(target_var)  # loss.item()을 사용하여 손실 값을 가져옴
        
#         loss += criterion(output, target_var[cc])
        
#     encoder.zero_grad()
#     decoder.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     return loss.data[0] / len(target_var)

In [51]:
def translate(enc_input = 'example.sentence.', predict_len = 100, temperature = 0.9):
    input_var = str2tensor(enc_input)
    encoder_hidden = encoder.init_hidden()
    encoder_outputs, encoder_hidden = encoder(input_var, encoder_hidden)
    
    hidden = encoder_hidden
    
    predicted = ''
    dec_input = str2tensor(SOS_token)
    for cc in range(predict_len):
        output, hidden = decoder(dec_input, hidden)
        
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        if top_i == EOS_token:
            break
            
        predicted_char = chr(top_i.item())
        predicted += predicted_char
        
        dec_input = str2tensor(predicted_char)
        
    return enc_input, predicted

In [52]:
encoder = sm.EncoderRNN(N_CHARS, HIDDEN_SIZE, N_LAYERS)
decoder = sm.DecoderRNN(HIDDEN_SIZE, N_CHARS, N_LAYERS)

if torch.cuda.is_available():
    decoder.cuda()
    encoder.cuda()
print(encoder, decoder)
test()

EncoderRNN(
  (embedding): Embedding(128, 128)
  (gru): GRU(128, 128)
) DecoderRNN(
  (embedding): Embedding(128, 128)
  (gru): GRU(128, 128)
  (out): Linear(in_features=128, out_features=128, bias=True)
)
tensor([[[ 0.1351,  0.0793,  0.2339,  0.3450, -0.3748,  0.1893,  0.0755,
          -0.3821, -0.1120, -0.3857,  0.0443, -0.1588,  0.0250, -0.1579,
          -0.1055,  0.3679,  0.1870,  0.1374, -0.5314,  0.4405,  0.2066,
           0.1151,  0.0812, -0.0775, -0.4187,  0.0508,  0.2170, -0.2497,
           0.2012, -0.1753,  0.1420, -0.3278, -0.1177, -0.0975, -0.2767,
          -0.0385, -0.0407, -0.5524,  0.0590, -0.2769, -0.2081,  0.2547,
           0.3303,  0.0886,  0.1457, -0.4725,  0.1074, -0.0768,  0.1089,
           0.1945, -0.0785, -0.4157, -0.3862, -0.4529,  0.5077,  0.2001,
           0.1002,  0.0997, -0.1742, -0.3428,  0.1969, -0.1498,  0.2929,
          -0.1584,  0.0887,  0.3462,  0.1456,  0.1522,  0.1640, -0.0381,
           0.3080, -0.0349, -0.2373, -0.1280, -0.3739,  0.2466, 

In [53]:
params = list(encoder.parameters()) + list(decoder.parameters())
optimizer = torch.optim.Adam(params, lr = 0.001)
criterion = torch.nn.CrossEntropyLoss()

train_loader = DataLoader(dataset = TextDataset(),
                         batch_size = BATCH_SIZE,
                         shuffle = True,
                         )

print("training for %d epochs..." % N_EPOCH)
for epoch in range(1, N_EPOCH + 1):
    for ii, (srcs, targets) in enumerate(train_loader):
        train_loss = train(srcs[0], targets[0])
        
        if ii % 100 == 0:
            print('[(%d %d%%) %.4f]' %
                  (epoch, epoch / N_EPOCH * 100, train_loss))
            print(translate(srcs[0]), '\n')
            print(translate(), '\n')

training for 20 epochs...
[(1 5%) 4.8377]
('godgrantwenevermayhaveneedofyou!', '(]\x1dl\x12,{Bq1r1D\x190\x0eRUB0foC\x05=Po^G<L\x07\x0cTcK.5j)jVT\x15\tO`\r?x/_C}\x08#!#B2U2g\t,@\x15\x0bQl\x0bgO\x08IGd-+\x1bMTkF_H\x7fYaHx\x7fjW\x003\x1eYx\x10') 

('example.sentence.', '@M\x18\x7f\x05\nR!t,M>7pz') 



C:\Users\10331\AppData\Local\Temp/ipykernel_16308/2324192785.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_char_index = torch.tensor(target_var[cc])


[(1 5%) 2.5302]
("myworkhathyetnotwarm'dme:fareyouwell:", 'I\x05O wed.') 

('example.sentence.', 'TIe oome qhou nagur') 

[(1 5%) 2.8125]
('thirdcitizen:', 'OOer llaat wonnt sis misacc in ofy gYes.') 

('example.sentence.', 'or me,') 

[(1 5%) 2.5918]
('inallhisowndesires;nay,lethimchoose', 'Bo the nisot dest inane thae brtnse,s thehou.') 

('example.sentence.', 'Mom of mo fe fous shrin faris ond isch, wer the by fors force eas the ons') 

[(1 5%) 2.3654]
('iwouldwithsuchperfectiongovern,sir,', 'Wourou, tond,') 

('example.sentence.', 'The Wouris bee,') 

[(1 5%) 2.1924]
('nowprisonertothepalsy,chastisethee', 'Wreat werast wis.irst wore dit.') 

('example.sentence.', 'Tdere,') 

[(1 5%) 1.9845]
('andyetnotmany.', 'Depy hesthawe theat ame delath') 

('example.sentence.', 'And your wiwlld.') 

[(1 5%) 2.5288]
('andgeorge,ofclarence:warwick,asourself,', "Lone herco comes'sn tarou heas hingst,") 

('example.sentence.', "Pet I my e'st an, bretinge, kor Byout the wCand thy zoudils foordst hi

[(1 5%) 2.7324]
('apothecary:', 'As grove thou know no take exwerger,') 

('example.sentence.', "Eform and unkingwen blood freate's the would") 

[(1 5%) 0.3548]
('warwick:', 'WARDID:') 

('example.sentence.', 'deasty;') 

[(1 5%) 0.2634]
('kingrichardii:', 'KING ERI EVINIO:') 

('example.sentence.', 'Metterime han butter') 

[(1 5%) 1.7410]
('now,lords,takeleaveuntilwemeetagain,', 'Not make be blodch a our my fatigst netction jay.') 

('example.sentence.', 'neter as stracts now stand other') 

[(1 5%) 1.4609]
('ifimightdiewithinthishour,ihavelived', 'I wones Ttrocire hime thut') 

('example.sentence.', 'my of washen,') 

[(1 5%) 1.5561]
('andbetruekingindeed,thoubuttheshadow.', 'And beds, will a mutly one cather') 

('example.sentence.', 'Eence, and your fare mory') 

[(1 5%) 1.7577]
('speak,goodcominius:', 'Sour Buck the mile bothy dose:') 

('example.sentence.', 'EDe mess tureers, genocker selve.') 

[(1 5%) 1.7021]
('anymanthatcanwritemayansweraletter.', 'And the all son, be good m

KeyboardInterrupt: 

# seq to seq with attention

In [2]:
from seq2seq_models import cuda_variable, str2tensor, EOS_token, SOS_token

N_LAYERS = 1
BATCH_SIZE = 3*64
N_EPOCH = 20
N_CHARS = 128  # ASCII
HIDDEN_SIZE = N_CHARS

In [3]:
def test():
    encoder_test = sm.EncoderRNN(10, 10, 2)
    decoder_test = sm.AttnDecoderRNN(10,10,2)
    
    if torch.cuda.is_available():
        encoder_test.cuda()
        decoder_test.cuda()
        
    encoder_hidden = encoder_test.init_hidden()
    word_input = cuda.variable(torch.LongTensor([1,2,3]))
    encoder_outputs, encoder_hidden = encoder_test(word_input, encoder_hidden)
    print(encoder_outputs,size())
    
    word_target = cuda_variable(torch.LongTensor([1,2,3]))
    decoder_attns = torch.zeros(1,3,3)
    decoder_hidden = encoder_hidden
    
    for cc in ragne(len(word_target)):
        decoder_output, decoder_hidden, decoder_attn = decoder_test(word_target[cc],
                                                                   decoder_hidden,
                                                                   encoder_outputs)
        
        print(decoder_output.size(), decoder_hidden.size(), decoder_attn.size())
        decoder_attns[0, cc] = decoder_attn.squeeze(0).cpu().data

In [4]:
def train(src, target):
    loss = 0 
    
    src_var = str2tensor(src)
    target_var = str2tensor(target, eos=True)
    
    encoder_hidden = encoder.init_hidden()
    encoder_outputs, encoder_hidden = encoder(src_var, encoder_hidden)
    
    hidden = encoder_hidden
    
    for cc in range(len(target_var)):
        
        token = target_var[cc - 1] if cc else str2tensor(SOS_token)
        output, hidden, attention = decoder(token, hidden, encoder_outputs)
        target_char_index = torch.tensor(target_var[cc])
        loss += criterion(output.view(1, -1), target_char_index.view(1))
        
        decoder_in = target_char_index
        
    encoder.zero_grad()
    decoder.zero_grad()
    loss.backward()
    optimizer.step()
        
    return loss.item() / len(target_var)  # loss.item()을 사용하여 손실 값을 가져옴

In [5]:
# Translate the given input
def translate(enc_input='example.sentence.', predict_len=100, temperature=0.9):
    input_var = str2tensor(enc_input)
    encoder_hidden = encoder.init_hidden()
    encoder_outputs, encoder_hidden = encoder(input_var, encoder_hidden)

    hidden = encoder_hidden

    predicted = ''
    dec_input = str2tensor(SOS_token)
    attentions = []
    for cc in range(predict_len):
        output, hidden, attention = decoder(dec_input, hidden, encoder_outputs)
        # Sample from the network as a multi nominal distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        attentions.append(attention.view(-1).data.cpu().numpy().tolist())

        # Stop at the EOS
        if top_i is EOS_token:
            break

        predicted_char = chr(top_i)
        predicted += predicted_char

        dec_input = str2tensor(predicted_char)

    return predicted, attentions

In [10]:
if __name__ == '__main__':
    
    encoder = sm.EncoderRNN(N_CHARS, HIDDEN_SIZE, N_LAYERS)
    decoder = sm.AttnDecoderRNN(HIDDEN_SIZE, N_CHARS, N_LAYERS)
    
    if torch.cuda.is_available():
        decoder.cuda()
        encoder.cuda()
    print(encoder, decoder)
    
    params = list(encoder.parameters()) + list(decoder.parameters())
    optimizer = torch.optim.Adam(params, lr = 0.001)
    criterion = torch.nn.CrossEntropyLoss()
    
    train_loader = DataLoader(dataset=TextDataset(),
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=2)
    
    print("Training for %d epochs..." % N_EPOCH)
    
    for epoch in range(1, N_EPOCH + 1):
        for ii, (srcs, targets) in enumerate(train_loader):
            train_loss = train(srcs[0], targets[0])
            
            if ii % 100 == 0 : 
                print('[(%d/%d %.2f%%) %.4f]' %
                      (epoch, N_EPOCH, (ii * len(srcs)) * 100 / (len(train_loader) * len(srcs)), train_loss))
#                 print('[(%d/%d %d%%) %.4f]' %
#                       (epoch, N_EPOCH, ii * len(srcs) * 100 / len(train_loader), train_loss)) 
#해당 코드 사용시 100% 넘어가서 위에로 수정함
                output, _ = translate(srcs[0])
                print(srcs[0], output, '\m')
                
                output, attentions = translate()
                print('example.sentence', output, '\n')

EncoderRNN(
  (embedding): Embedding(128, 128)
  (gru): GRU(128, 128)
) AttnDecoderRNN(
  (attn): Linear(in_features=128, out_features=128, bias=True)
  (embedding): Embedding(128, 128)
  (gru): GRU(128, 128, dropout=0.1)
  (out): Linear(in_features=256, out_features=128, bias=True)
)
Training for 20 epochs...


C:\Users\10331\AppData\Local\Temp/ipykernel_18484/1417949097.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_char_index = torch.tensor(target_var[cc])


[(1/20 0.00%) 4.8780]
dukevincentio: ^iiL,wQ3.J%;mPxolOo<Fg8<@2to^m{92^maO33CyU#DL
XqI{5$uDYPO|,Q@7H-4f >W \m
. >v@;_Z2 ,f65kv7`:%b`k9 FYIKz|nQSJ9O4}GUp&+~	

[(1/20 58.48%) 2.7747]
thegodsforbid! phts to , dorn,r oor faes coo ta ghuwiwercccWe ss t ayt cekrtso sor ara, fi he ve t cren hceusTo!  \m
example.sentence nhe tpger ofd ous o  olee y henc aidinfu tic iyrcn aher t tero nryou tesecocheP,txSWI te he  oo 

[(2/20 0.00%) 2.4903]
whichne'ercamefromthelungs,buteventhus-- Whaade	s thie, uaitsthes,e nedegy ust som toy veuCke ingofesty lch ;oyche'nd!' sind- an, -u sgof \m
example.sentence SLor:O meayecl, ind ther heved fatfnt saand ofeas have.hve th hweshqincde, her these cong anos: 

[(2/20 58.48%) 2.4151]
cansetthedukeupindespiteofme. Hwe hed the s hithr aor thaves hes thoand ang he'ne whe, sies dond ithse wat  tofuw< bur knesr'd dii \m
example.sentence He row stor hen oll, and. onl mes pont cend Rpays slo wer, dir co wast ark lore

example.sentence Ex mle tencie... ENce. ...ble. iplessence.... ce.. ENce....... Imple... 

[(16/20 58.48%) 0.5316]
andforthymaintenancecommitshisbody And for thy maintencecamitss bodyd yod sod yodod yorthy maits bodds body yor mitsb odyd yor \m
example.sentence ExU mle serce..'mple jample.NCE.En ce.ze..ble.rce.. A mle. I mle.nce.... 

[(17/20 0.00%) 0.2372]
andwringtheawfulsceptrefromhisfist, And wring theaw fulsceptrefromhis fist, st, cepptrefrom his fist, peptrefromhis fist,, cepe pt \m
example.sentence Edextence.nce?.nce.nce.. EnCe...nce,n ce. NCETEEN CENTence. I Plence.. bl 

[(17/20 58.48%) 0.2545]
youhaveundoneamanoffourscorethree, You have und on eam anof four scoreth Ree,, hore,,n of four scoreth tree,, hree,,e, hre \m
example.sentence extexce ce. sence..nce.N. EN RETn ce. Ble.snce..nce...nce. sence..nce.t ENce. 

[(18/20 0.00%) 1.2312]
friarlau